# Sustainable Sourcing Layers in a Cloud Function

This notebook demonstrates deployment of Google Cloud functions to get commodity and forest information in a format designed to be integrated to other workflows.

The 2025a release of Forest Data Partnership sustainable sourcing layers including palm, rubber, cocoa and coffee are used for commodity information.  For details on how these layers were produced, see [the technical documentation on GitHub](https://github.com/google/forest-data-partnership/tree/main/models).  In particular, see [the limitations](https://github.com/google/forest-data-partnership/tree/main/models#limitations).  See also the [Forest Data Partnership publisher catalog](https://developers.google.com/earth-engine/datasets/publisher/forestdatapartnership) for dataset descriptions.  See [this Earth Engine Code Editor script](https://goo.gle/fodapa-layers) for a demonstration of how choice of thresholds affects the mapped results.

Note that users of commercial projects will need to request access to the Forest Data Partnership datasets with [this form](https://docs.google.com/forms/d/e/1FAIpQLSe7L3eh6t2JIPqEtAQwXwY7ZmW52v8W5vrIi4QN_XYgTNJZLw/viewform).

**WARNING**: These demos consume billable resources and may result in charges to your account!

# Setup

## Set the Colab Environment environment variables

Import the Python libraries used by the section.

In [ ]:
import json
import os

from google.colab import userdata

This notebook assumes that the GCP project and compute region information are stored as Colab secrets. This avoids storing user-specific and potentially sensitive information in the notebook. See this guide for information on how to use Colab secrets.

In [ ]:
# Load from Colab secrets
PROJECT = userdata.get('EE_PROJECT_ID')
REGION = userdata.get('GCP_REGION')

# Store as environment variables
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

## Authenticate

Authenticate using the Google Cloud CLI (gcloud).

In [ ]:
!gcloud auth login --project {PROJECT} --billing-project {PROJECT} --update-adc

## Load WHISP example data

We will test out the Cloud Run function using some example data from
[WHISP](xhttps://openforis.org/solutions/whisp/).

Start by downloading some WHISP example data from GitHub and convert it to an `ee.FeatureCollection`.

In [ ]:
fc_list = !curl https://raw.githubusercontent.com/forestdatapartnership/whisp/main/tests/fixtures/geojson_example.geojson
fc_obj = json.loads("\n".join(fc_list))
features = fc_obj['features']

Print an example feature.

In [ ]:
features[0]

Extract the feature geometries.

In [ ]:
geoms = [f['geometry'] for f in features]

Print an example geometry.


In [ ]:
geoms[0]

# Create the Cloud Run function

## Checkout the Cloud Run function source code

Cloud Run functions are defined from a collection of files in a [source directory](https://cloud.google.com/run/docs/write-functions#directory_structure). This section downloads Cloud Run function definition files to Colab's storage using git's [sparse-checkout](https://git-scm.com/docs/git-sparse-checkout).

In [ ]:
%%bash
is_git_repo() {
    git rev-parse --is-inside-work-tree >/dev/null 2>&1
}

if is_git_repo; then
  echo "This is already a git repository. No action was taken."
else
  echo "This is not a git repository. Initiating sparse checkout..."
  repo_url=https://github.com/tylere/forest-data-partnership.git
  branch=cloud-function-folder
  cloud_function_source_dir=cloud_functions/sustainable_sourcing_layers

  # Use git sparse-checkout
  git config --global init.defaultBranch $branch
  git init
  git remote add origin $repo_url
  git sparse-checkout init --cone
  git sparse-checkout set $cloud_function_source_dir
  git pull origin $branch
fi

## Test the Cloud Run function code

Import the code and run a few tests locally.

In [ ]:
from pprint import pprint
import ee

print('Using EE version: ', ee.__version__)

In [ ]:
from cloud_functions.sustainable_sourcing_layers import suso_layers_2025a

# Check the image metadata and bands.
layers = suso_layers_2025a.get_suso_layers_2025a()
pprint(layers.getInfo())

Sample the layers at a specified point.

In [ ]:
test_point = ee.Geometry.Point(104.33, -3.41)
pprint(layers.reduceRegion(ee.Reducer.mean(), test_point, 10).getInfo())

Get the class areas at a specific point

In [ ]:
print(
    suso_layers_2025a
        .get_areas_image()
        .reduceRegion(
            ee.Reducer.mean(),
            test_point,
            10
          ).getInfo()
)

Get statistics within an example geometry.

In [ ]:
suso_layers_2025a.get_suso_stats(geojson=geoms[0])

## Deploy the Cloud Run function

See [this quickstart](https://cloud.google.com/run/docs/quickstarts/functions/deploy-functions-gcloud) for details on deploying Cloud Run functions.

The Cloud Run function configuration files are stored in the folder `cloud_functions/sustainable_sourcing_layers`.

Inspect the list of Python packages required to run the Cloud Run function.

In [ ]:
%cat cloud_functions/sustainable_sourcing_layers/requirements.txt

## Use gcloud to Deploy the Cloud Run function

Use the following `gcloud` command to deploy the Cloud Run function.

*Note that the Compute Engine default service account is being used for authentication to Earth Engine.  For commercial access to the sustainable sourcing layers, ensure that the service account is approved for commercial access ([request form](https://docs.google.com/forms/d/e/1FAIpQLSe7L3eh6t2JIPqEtAQwXwY7ZmW52v8W5vrIi4QN_XYgTNJZLw/viewform)).*

In [ ]:
!gcloud functions deploy 'suso_function' \
  --gen2 \
  --region={REGION} \
  --project={PROJECT} \
  --runtime=python312 \
  --source='cloud_functions/sustainable_sourcing_layers' \
  --set-env-vars PROJECT={PROJECT} \
  --entry-point=main \
  --trigger-http \
  --no-allow-unauthenticated \
  --timeout=300s

 It's helpful to follow the link in the deployment output to monitor and/or debug your Cloud Run function.  In particular, see the metrics and logs tabs to help resolve perfpormance issues.

# Call the deployed Cloud Run function

This section demonstrates how to call the deployed Cloud Run function using different tools.

*(Note that after creating a new Colab runtime you must first run the [Setup](#scrollTo=yaw0K_p2ieqt) section before running this section.)*

## Call using curl

The section demonstrates calling Cloud Run function using [curl](https://curl.se/), a general purpose command-line tool used to transfer data to or from a server using a wide range of network protocols.

Create a JSON formatted string of the Cloud Run function inputs (i.e. WHISP sample geometries).

In [ ]:
payload = {
    "calls": [[json.dumps(g)] for g in geoms]
}
payload_json = json.dumps(payload, separators=(',', ':'))
payload_json

Format the payload for curl, by wrapping it in escaped single quotes.

In [ ]:
post_data = f"'{payload_json}'"
post_data

Use curl to submit a test POST data. This might take a few seconds to execute.

In [ ]:
response_body_text_list = !curl -X POST https://{REGION}-{PROJECT}.cloudfunctions.net/suso_function \
  -H "Authorization: bearer $(gcloud auth print-identity-token)" \
  -H "Content-Type: application/json" \
  -d {post_data}
response_body_text_list

Use Python to parse and inspect the Cloud Run function response.

In [ ]:
response = response_body_text_list[0]
response_json = json.loads(response)
replies_json = [json.loads(reply) for reply in response_json['replies']]
reply_keys = replies_json[0].keys()

print(f'{len(replies_json) = }')
print(f'{reply_keys = }')

Pretty print the replies.

In [ ]:
print(json.dumps(replies_json, indent=4))

## Call using Python

In [ ]:
import google.auth.transport.requests
import requests

Retrieve your Google Cloud account identity token.

In [ ]:
# Set this to the target audience (Cloud Run service URL)
audience = "https://us-west1-vorgeo-training.cloudfunctions.net/suso_function"

project = os.environ['PROJECT']
credentials, _ = google.auth.default(
    scopes=['https://www.googleapis.com/auth/earthengine'],
)
credentials.refresh(
    request = google.auth.transport.requests.Request()
)
identity_token = credentials.id_token

print(f'{identity_token = }')

Make a call to the Cloud Run function.

In [ ]:
headers = {
    "Authorization": f"Bearer {identity_token}",
    "Content-Type": "application/json"
}

# Prepare the payload in the format expected by the Cloud Function
payload = {
    "calls": [[json.dumps(g)] for g in geoms]
}

response = requests.post(audience, headers=headers, json=payload)
print(response.text)

Parse the Cloud Run function response.

In [ ]:
response_json = json.loads(response.text)
replies_json = [json.loads(reply) for reply in response_json['replies']]
print(json.dumps(replies_json, indent=4))